In [ ]:
import gspread
import pandas as pd
import requests
import geopy.distance
import numpy as np
from datetime import timedelta
import time
import json
from itertools import chain

In [ ]:
gc = gspread.oauth()

In [ ]:
sh = gc.open_by_key('1bKaLOU0vsS114xZVPG8bRNQT2lWiPsqj7HJ7N53z4FM')

In [ ]:
ws = sh.worksheet("Página1")

In [ ]:
df = pd.DataFrame(ws.get_all_records())

In [ ]:
df['Latitude'] = df['Latitude'].astype("string")
df['Longitude'] = df['Longitude'].astype("string")

In [ ]:
df['Latitude'] = df['Latitude'].replace("@", ".", regex=True)
df['Longitude'] = df['Longitude'].replace("@", ".", regex=True)

In [ ]:
exluirLinhas = df[df['Longitude']=='NULL'].index
df=df.drop(exluirLinhas)
exluirLinhas = df[df['Latitude']=='NULL'].index
df=df.drop(exluirLinhas)

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df_directions = df[['NumeroSerie','Latitude','Longitude']].drop_duplicates()

In [ ]:
listagem_base = df_directions.values.tolist()
listagem_base

In [ ]:
df_gold=[]
list_infos=[]

#API GOOGLE
#Como pegar dados do google api 
url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=45.7597%2C4.8422&destinations=48.8567%2C2.3508&key="

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

res_json = (response.text)
res_final = json.loads(res_json)

EnderecoDestino = res_final['destination_addresses'][0]
EnderecoOrigem = res_final['origin_addresses'][0]
Distance_text = res_final["rows"][0]['elements'][0]['distance']['text']
Distance_value = res_final["rows"][0]['elements'][0]['distance']['value']
durantion_text = res_final["rows"][0]['elements'][0]['duration']['text']
durantion_value = res_final["rows"][0]['elements'][0]['duration']['value']

print(EnderecoDestino)
print(EnderecoOrigem)
print(Distance_text)
print(Distance_value)
print(durantion_text)
print(durantion_value)

list_infos = [EnderecoDestino,EnderecoOrigem,Distance_text,Distance_value,durantion_text,durantion_value]

print(list_infos)

In [ ]:
#API GOOGLE
def buscaGoogle(geoOrigem,geoDestino):
    
    print("Sera tratado origem: "+str(geoOrigem))
    print("Sera tratado destino: "+str(geoDestino))
    
    list_infos = []
    
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+geoOrigem[0]+"%2C"+geoOrigem[1]+"&destinations="+geoDestino[0]+"%2C"+geoDestino[1]+"&key="
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    res_json = (response.text)
    res_final = json.loads(res_json)
    
    verifica_status = res_final["rows"][0]['elements'][0]['status']
    
    print(verifica_status)
    
    print(res_final)
    print("\n-------------------")

    EnderecoDestino = res_final['destination_addresses'][0]
    EnderecoOrigem = res_final['origin_addresses'][0]
    Distance_text = res_final["rows"][0]['elements'][0]['distance']['text']
    Distance_value = res_final["rows"][0]['elements'][0]['distance']['value']
    durantion_text = res_final["rows"][0]['elements'][0]['duration']['text']
    durantion_value = res_final["rows"][0]['elements'][0]['duration']['value']
    
    list_infos = [EnderecoDestino,EnderecoOrigem,Distance_text,Distance_value,durantion_text,durantion_value]
    
    #time.sleep(1)
    
    return list_infos

In [ ]:
def CalcListagem(radar, lat,long):
    for i in listagem_base:
        if i[0] != radar:
            geoOrigem=[lat,long]
            geoDestino=[i[1],i[2]]
            dis = buscaGoogle(geoOrigem,geoDestino)
            if dis != 0 and dis is not None:
                df_gold.append(
                    {
                        'RadarBase': radar,
                        'LatitudeBase': lat,
                        'LongitudeBase':  long,
                        'RadarComparacao':i[0],
                        'LatitudeComparacao':i[1],
                        'LongitudeComparacao':i[2],
                        'EnderecoDestino':dis[0],
                        'EnderecoOrigem':dis[1],
                        'DistanciaTexto':dis[2],
                        'Distanciavalor':dis[3],
                        'DuracaoTexto':dis[4],
                        'DuracaovAlor':dis[5]
                    }
                            )
                #print("RadarBase: "+str(df_gold))
                #print("\n")
    return True

In [ ]:
df_gold

In [ ]:
for x in listagem_base:
    print(x)
    
    CalcListagem(x[0],x[1],x[2])
        
df_directions_Gold = pd.DataFrame(df_gold)

In [ ]:
df_directions_Gold = pd.DataFrame(df_gold)
df_directions_Gold

In [ ]:
df_realizado = df_directions_Gold[['RadarBase','LatitudeBase','LongitudeBase','RadarComparacao','LatitudeComparacao','LongitudeComparacao']]

listagem_realizado = df_realizado.values.tolist()

In [ ]:
for base in listagem_realizado:
    if base[3]== 'BRI1286':
        
        print(base)
        print(base[0])
        print(base[3])

In [ ]:
df_gold2=[]

In [ ]:
def CalcListagem_fase2(radar, lat,long):
    for i in listagem_base:
        contador=0
        if i[0] != radar:
            for real in listagem_realizado:           
                if radar == real[0] and i[0] ==real[3]:
                    print(str(radar)+" e "+str(i[0]))
                    #print("Já foi inserido na lista")
                    contador=1
            if contador == 0:
                print("Não está na listagem, puxando do google")
                geoOrigem=[lat,long]
                geoDestino=[i[1],i[2]]

                dis = buscaGoogle(geoOrigem,geoDestino)
                if dis != 0 and dis is not None:
                    df_gold2.append(
                        {
                            'RadarBase': radar,
                            'LatitudeBase': lat,
                            'LongitudeBase':  long,
                            'RadarComparacao':i[0],
                            'LatitudeComparacao':i[1],
                            'LongitudeComparacao':i[2],
                            'EnderecoDestino':dis[0],
                            'EnderecoOrigem':dis[1],
                            'DistanciaTexto':dis[2],
                            'Distanciavalor':dis[3],
                            'DuracaoTexto':dis[4],
                            'DuracaovAlor':dis[5]
                        }
                                )
    return True

In [ ]:
for x in listagem_base:
    CalcListagem_fase2(x[0],x[1],x[2])
        
df_directions_Gold2 = pd.DataFrame(df_gold2)

In [ ]:
df_directions_Gold2 = pd.DataFrame(df_gold2)

In [ ]:
df_directions_Gold2.info()

In [ ]:
df_directions_Gold2

In [ ]:
frames = [result, df_directions_Gold2]

df_final = pd.concat(frames)

In [ ]:
df_final.drop_duplicates()

In [ ]:
result.drop_duplicates()

In [ ]:
df_realizado = result[['RadarBase','LatitudeBase','LongitudeBase','RadarComparacao','LatitudeComparacao','LongitudeComparacao']]

listagem_realizado = df_realizado.values.tolist()

In [ ]:
df_realizado

In [ ]:
df_final.info()

In [ ]:
df_final.to_csv('bh_trans_distancias_google.csv', sep=';',encoding='utf-8')